# Week 10 Assignment

**Due date**: Tue, Oct 27 by 3:30pm (submit on Sakai)

# Building a bioinformatics pipeline, part I

The goal of this assignment is to create the basis of a bioinformatics pipeline that can be used to study the variation and evolution of genes/proteins.  The pipeline you will build will accomplish the following steps:

1. Read in a CSV delimited spreadsheet containing Genbank accession numbers

2. Use Biopython to download to Genbank files corresponding to the accession numbers in step 1

3. Extract the protein sequences of the genes represented in each genbank file, saving the results into a single FASTA file

4. Align the sequences with the multiple sequence alignment program MUSCLE using the appropriate interface from Biopython

## Data set of interest

We will use our pipeline to study interspecific variation in betacoronavirus spike proteins, and the protein that the spike proteins recognize and bind to on mammalian cells, called ACE2 (Angiotensin-Converting Enzyme 2). For details on background read the following papers:

1. Hu, B., Guo, H., Zhou, P. et al. Characteristics of SARS-CoV-2 and COVID-19. Nat Rev Microbiol (2020). https://doi.org/10.1038/s41579-020-00459-7


2. Broad and Differential Animal Angiotensin-Converting Enzyme 2 Receptor Usage by SARS-CoV-2
Xuesen Zhao, Danying Chen, Robert Szabla, Mei Zheng, Guoli Li, Pengcheng Du, Shuangli Zheng, Xinglin Li, Chuan Song, Rui Li, Ju-Tao Guo, Murray Junop, Hui Zeng, Hanxin Lin
Journal of Virology Aug 2020, 94 (18) e00940-20; https://doi.org/10.1128/JVI.00940-20


In [192]:
import pandas as pd
import os
from Bio import Entrez
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature
from Bio.SeqRecord import SeqRecord

# Problems

## 1. Reading list of accessions [5 pts]

Write a function to read a CSV formatted file that contains names and accession numbers for sequences of interest.  Each file will have two columns, "Name" and "Accession", giving the names of the organism that the sequence comes from and the corresponding Genbank accession numbers for each sequence. 

Your function should return a dictionary, with the names (strings) as the keys and the accession numbers (strings) as the values.

The following files give accession list for a set of coronavirus spike proteins and a set of mammalian ACE2 genes:

* `ACE2-accessions.csv`
* `Spike-accessions.csv`

In [193]:
def read_accessions(csvfile):
    df = pd.read_csv(csvfile)
    listName = df.Name.tolist()
    listAccession = df.Accession.tolist()
    retdict = dict(zip(listName,listAccession))
    return retdict

In [194]:
# The following tests should work if your read_accessions function works correctly

spikedict = read_accessions("Spike-accessions.csv")
assert(spikedict["RaTG13"] == "MN996532.1")
print(spikedict)

ace2dict = read_accessions("ACE2-accessions.csv")
assert(ace2dict["Hog badger"] == "MT663962")
print(ace2dict)

{'SARS-CoV-2': 'MN908947', 'SARS-CoV BJ01': 'AY278488', 'RaTG13': 'MN996532.1', 'GX pangolin': 'MT040333.1', 'Bat WIV1': 'KF367457', 'Bat ZC45': 'MG772933.1', 'SARS-CoV Tor2': 'NC_004718.3'}
{'Domestic dog': 'MT663955', 'Mexican free-tailed bat': 'MT663956', 'Chinese ferret badger': 'MT663957', 'Raccoon dog': 'MT663958', 'Domestic cat': 'MT663959', 'Rhesus monkey': 'MT663960', 'New Zealand White rabbit': 'MT663961', 'Hog badger': 'MT663962', 'Platypus': 'XM_001515547', 'Human': 'NM_001371415.1', 'Civet': 'AY881174.1', 'Rat': 'NM_001012006.1', 'Chinese horseshoe bat': 'KC881004.1', 'Pangolin': 'XM_017650263.1', 'Mouse': 'NM_001130513'}


## 2. Downloading Genbank files [7.5 pts]

Biopython includes facilities for automatically downloading Genbank files from the web, using NCBI's Entrez interface, as described in [Chap 5](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec59) and [Chap 9](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec%3Aefetch) of the Biopython tutorial.  

**IMPORTANT** Before working on this problem make sure to read the ["Entrez Guidelines"](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec%3Aentrez-guidelines) about how to use the Entrez resources responsibly (if you don't do this you risk having your IP address banned from accessing Entrez). In particular make sure your specify your email address for validating Entrez access after importing the Etnrez library.

``` 
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"  # replace with your Duke email
```

Write a function that takes as input an accession dictionary (e.g. created by the `read_accessions` function). For each accession, the function should use Biopython's Entrez tools to download the accessions of interest as genbank files and then each accession out to your file system, with file names formatted as `accession_number.gb`.  

Your function should return a dictionary relating the names in the accession dictionary to the filenames of the genbank files.  For examp


In [195]:
def download_accessions(accessiondict):
    #list of values
    #access all values from Entrez
    #make dictionary of values : file names as accession_number.gb
    
    fname = ""
    listValueFiles = []
    for i in accessiondict.values():
        Entrez.email = "ashrit.verma@duke.edu"  # Always tell NCBI who you are
        filename = i+'.gb'
        listValueFiles.append(filename)
        
        if not os.path.isfile(filename):
            # Downloading...
            net_handle = Entrez.efetch(
                db="nucleotide", id=i, rettype="gb", retmode="text"
            )
            out_handle = open(filename, "w")
            out_handle.write(net_handle.read())
            out_handle.close()
            net_handle.close()
            print("Saved")
        print("Parsing...")
        record = SeqIO.read(filename, "genbank")
        print(record)
    
    dictret = dict(zip(accessiondict.keys(),listValueFiles))
    return dictret

In [196]:
# test your function
testdict = {"Human":"NM_001371415.1"}

testfiledict = download_accessions(testdict)
assert(testfiledict["Human"] == "NM_001371415.1.gb")
print(testfiledict)
# Note that your test should also write a file with the name NM_001371415.1.gb in the directory in your working directory

Parsing...
ID: NM_001371415.1
Name: NM_001371415
Description: Homo sapiens angiotensin I converting enzyme 2 (ACE2), transcript variant 1, mRNA
Number of features: 41
/molecule_type=mRNA
/topology=linear
/data_file_division=PRI
/date=04-OCT-2020
/accessions=['NM_001371415']
/sequence_version=1
/keywords=['RefSeq', 'MANE Select']
/source=Homo sapiens (human)
/organism=Homo sapiens
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
/references=[Reference(title='Angiotensin-converting enzymes (ACE, ACE2) gene variants and COVID-19 outcome', ...), Reference(title='Most frequent South Asian haplotypes of ACE2 share identity by descent with East Eurasian populations', ...), Reference(title='Analysis of ACE2 genetic variants in 131 Italian SARS-CoV-2-positive patients', ...), Reference(title='Mass Spectrometry and Structural Biology Techniques in the 

Helper function to create Genbank files containing ONLY the spike protein

In [197]:
def filter_spike_cds(spikeaccessiondict):
    newdict = {}
    for key,value in spikeaccessiondict.items():
        rec = SeqIO.read(value, format="gb")
        spikeftr = None
        for ftr in rec.features:
            if ftr.type == "CDS":
                if "gene" in ftr.qualifiers:
                    if "S" in ftr.qualifiers["gene"]:
                        spikeftr = ftr
                        break
                elif "product" in ftr.qualifiers:
                    for descriptor in ftr.qualifiers["product"]:
                        if "spike" in descriptor:
                            spikeftr = ftr
                            break
        if spikeftr is not None:
            newrec = SeqRecord.SeqRecord(rec.seq, id=rec.id,name=rec.name,
                                         description=rec.description, features=[spikeftr],
                                         annotations=rec.annotations)
            fname = rec.id + "-spikeonly.gb"
            SeqIO.write(newrec, fname, format="gb")
            newdict[key] = fname
    return newdict

## 3. Creating FASTA files [7.5 pts]

Your next two functions will take the dictionary returned from the `download_accessions` function and then read each file, extracting either the nucleotide sequence or the  translations of the primary coding sequences (CDS) in each file, and will then write out a *single* FASTA file (`fastaname`) with the protein sequences for all the sequences of interest. After parsing the genbank files and writing the corresponding FASTA file, your functions should return the filename of the FASTA file it created.

In [198]:
def genbank2fasta_nucleotide(accessionfiledict, fastaname):
    sequences = []
    for key,value in accessionfiledict.items():
        input_seq_iterator = SeqIO.parse(value, "genbank")
        array1 = list(input_seq_iterator)
        listftr = [ftr for ftr in array1[0].features if ftr.type == 'CDS']
        if 0 < len(listftr):
            f = listftr[0]
            x = f.extract(array1[0].seq)
            simple_seq = SeqRecord(x, id=value, name=key)
            sequences.append(simple_seq)
        savedir = "C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/" + fastaname
        SeqIO.write(sequences, savedir, 'fasta')
    return (fastaname)
genbank2fasta_nucleotide(testfiledict, 'allnucleotidesequences')

'allnucleotidesequences'

In [199]:
def genbank2fasta_protein(accessionfiledict, fastaname):
    
    #simple_seq is coding_dna
    #coding_dna.translate()
    sequences = []
    for key,value in accessionfiledict.items():
        input_seq_iterator = SeqIO.parse(value, "genbank")
        array1 = list(input_seq_iterator)
        listftr = [ftr for ftr in array1[0].features if ftr.type == 'CDS']
        if 0 < len(listftr):
            f = listftr[0]
            x = f.extract(array1[0].seq)
            simple_seq = SeqRecord(x, id=value, name=key)
            translated_seq = simple_seq.translate()
            sequences.append(translated_seq)
        savedir = "C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/" + fastaname
        SeqIO.write(sequences, savedir, 'fasta')
    return (fastaname)
genbank2fasta_protein(testfiledict, 'allproteinsequences')

'allproteinsequences'

## 4. Aligning protein sequences using MUSCLE [5 pts]

[MUSCLE](http://www.drive5.com/muscle/) is a popular multiple sequence alignment program.  You can download a command line version of MUSCLE at that link, and "install" it by unzipping the binary and placing it somewhere in your path (I will show you how to do this in class).

Once MUSCLE is on your computer you should test MUSCLE "by hand" (we'll do this in class).  Biopython also provides an interface to interact with MUSCLE from within a Python pipeline, as described in [Section 6.5](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec92) of the Biopython tutorial.

Using the `MuscleCommandLine` interface that Biopython provides, write a function that aligns the amino acid sequences in a FASTA file:

In [200]:
from Bio.Align.Applications import MuscleCommandline
from io import StringIO
from Bio import AlignIO

In [201]:
def align_via_MUSCLE(infastafile, outfastafile):
    muscle_exe = "C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/muscle3.8.31.exe"
    in_file = "C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/" + infastafile
    out_file = "C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/" + outfastafile
    cline = MuscleCommandline(muscle_exe, input=in_file,out=out_file)
    cline()
    return(cline)

In [202]:
align_via_MUSCLE(genbank2fasta_protein(testfiledict, 'allproteinsequences'), "Test-MUSCLE-Output.fasta")

MuscleCommandline(cmd='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/muscle3.8.31.exe', input='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/allproteinsequences', out='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/Test-MUSCLE-Output.fasta')

## 5. Test the entire pipeline [5 pts]

If each of your functions above works, then the following small code block implements the entire pipeline from start to finish:

In [206]:
spikedict = read_accessions("Spike-accessions.csv")
spikefiledict = download_accessions(spikedict)
spikeonlydict = filter_spike_cds(spikefiledict)
spikenucfasta = genbank2fasta_nucleotide(spikeonlydict, "Spike-nucs.fasta")
spikeprotfasta = genbank2fasta_protein(spikeonlydict, "Spike-prot.fasta")
align_via_MUSCLE(spikeprotfasta, "Spike-prot-aligned.fasta")

Parsing...
ID: MN908947.3
Name: MN908947
Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Number of features: 23
/molecule_type=ss-RNA
/topology=linear
/data_file_division=VRL
/date=18-MAR-2020
/accessions=['MN908947']
/sequence_version=3
/keywords=['']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
/references=[Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Direct Submission', ...)]
/comment=On Jan 17, 2020 this sequence version replaced MN908947.2.
/structured_comment=OrderedDict([('Assembly-Data', OrderedDict([('Assembly Method', 'Megahit v. V1.1.3'), ('Sequencing Technology', 'Illumina')]))])
Seq('ATTA

MuscleCommandline(cmd='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/muscle3.8.31.exe', input='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/Spike-prot.fasta', out='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/Spike-prot-aligned.fasta')

In [204]:
ace2dict = read_accessions("ACE2-accessions.csv")
ace2filedict = download_accessions(ace2dict)
ace2nucfasta = genbank2fasta_nucleotide(ace2filedict, "Ace2-nucs.fasta")
ace2protfasta = genbank2fasta_protein(ace2filedict, "Ace2-prot.fasta")
align_via_MUSCLE(ace2protfasta, "Ace2-prot-aligned.fasta")

Parsing...
ID: MT663955.1
Name: MT663955
Description: Canis lupus familiaris angiotensin I converting enzyme 2 mRNA, complete cds
Number of features: 2
/molecule_type=mRNA
/topology=linear
/data_file_division=MAM
/date=10-JUL-2020
/accessions=['MT663955']
/sequence_version=1
/keywords=['']
/source=Canis lupus familiaris (dog)
/organism=Canis lupus familiaris
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Laurasiatheria', 'Carnivora', 'Caniformia', 'Canidae', 'Canis']
/references=[Reference(title='Broad and differential animal ACE2 receptor usage by SARS-CoV-2', ...), Reference(title='Direct Submission', ...)]
/structured_comment=OrderedDict([('Assembly-Data', OrderedDict([('Sequencing Technology', 'Sanger dideoxy sequencing')]))])
Seq('ATGTCAGGCTCTTCCTGGCTCCTTCTCAGCCTTGCTGCTCTAACTGCTGCTCAA...TAG', IUPACAmbiguousDNA())
Parsing...
ID: MT663956.1
Name: MT663956
Description: Tadarida brasiliensis angiotensin I converting en

MuscleCommandline(cmd='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/muscle3.8.31.exe', input='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/Ace2-prot.fasta', out='C:/Users/cleve/OneDrive/Documents/Notes/Notebooks Linked Materials/Duke University/Bio 208 Computing on the Genome/Assignment Week 10/Ace2-prot-aligned.fasta')